# Encode - Harmonizome

In [401]:
#!/usr/bin/python

# Packages and dependencies
from ipywidgets import *
import pandas as pd
import glob
import numpy as np
import os
import re
import json
import requests
import csv
import requests
import urllib2
import StringIO
import gzip
import matplotlib.pyplot as plt
import scipy.spatial.distance as dist

# Metadata

In [402]:
df_meta = pd.read_csv('/Users/Carol/Desktop/Encode_eu/mm9_metadata.csv', header = 0)
df_meta.head(5)
df_meta.shape

(193, 48)

In [403]:
# df_meta ["Biosample term id"]


In [404]:
# list(df_meta)

In [405]:
# Checking columns names
# list(df_meta)

In [406]:
# dataframe ENCODE_ID to enrichr_id
df_meta = pd.read_csv('/Users/Carol/Desktop/Encode_eu/mm9_metadata.csv', header = 0)
df_enrich = df_meta["Experiment_target"].map(str) + '_' + df_meta["Biosample term name"].map(str) + '_'+ df_meta["Assembly"]
df_ex = df_meta["Experiment accession"]
df_tf = df_meta["Experiment_target"]
df_metadata = pd.concat((df_ex, df_tf,df_enrich), axis=1)

df_metadata.head(3)
# df_metadata.to_csv('Experiment.csv')

,Experiment accession,Experiment_target,0
0,ENCSR000EQW,ZKSCAN1,ZKSCAN1_CH12.LX_mm9
1,ENCSR000ERC,BHLHE40,BHLHE40_CH12.LX_mm9
2,ENCSR000EQQ,ZC3H11A,ZC3H11A_CH12.LX_mm9


In [407]:
# df_enrich = df_meta[df_meta["Experiment_target"].replace({'-mouse': ''}, regex=True)].map(str) + df_meta["Biosample term name"].map(str) + '_'+ df_meta["Assembly"]
# df_enrich

# Gene Set Library

In [357]:
## NEW FOLDER UCSC example

import glob
import numpy as np
path = '/Users/Carol/Desktop/EncodeUCSC/bedFiles/' ## NOVA pasta

files = glob.glob(os.path.join(path, '*.txt')) # All txt files in folder
fids = np.unique([file.split('_')[0] for file in files])  ## unique accession IDs

for fid in fids:
    df_ = pd.DataFrame()
    for fname in files:
        if fid in fname:
            df = pd.read_csv(fname, sep='\t', header=0)
            df.columns = ['ID ', 'Chr', 'Start', 'End', 'Strand', 'Peak Score', 'Focus Ratio/Region Size', 'Annotation', 'Detailed Annotation', 'Distance', 'PromoterID', 'EntrezID', 'Unigene', 'Refseq', 'Ensembl', 'GeneName', 'GeneAlias','GeneDescription', 'GeneType']
            
            df_ = pd.concat((df_, df))
    df_ = df_.groupby('GeneName').agg(np.mean) # group duplicated genes by the mean
    # Here groupby is combining the results into a data structure based on the mean . groupby gene name than aplly the the function mean 
    #to the resulting groups
    df = df_[(df_.Distance <= 2000) & (df_.Distance >= -2000)] # Use a single column's value to select data
    #reindexing allows you to add/change/delete the index on a specified axis
    #this returns a copy of the data
    df = df.reindex(df.Distance.abs().sort_values(inplace=False, ascending=True).index) #sorts by abs value, then reorganizes gene column with the index
    df_2000 = df[:3000]
    df_2000.to_csv(fid,sep='\t')

# GENE MATRIX

get all the files together

In [358]:
# Each iteration creats one data frame per experiment. This folder has been renamed annotatedPeaks 
# for clarity porpuses.
# df_2000.head(5)

In [419]:
# if for some reason not finding GeneName on index, delete DS store file from folder
import os

files = os.listdir("/Users/Carol/Desktop/annotatedPeaks")
# print files

df_ = pd.DataFrame()

for f in files:
    fname = "/Users/Carol/Desktop/annotatedPeaks/"+f
    for f in f:
        df = pd.read_csv(fname, sep='\t', header=0)
        df = df[['GeneName']]
#         df= df.rename(columns = {'GeneName':fname})
#         df = df.T
    df_ = pd.concat((df_, df))

df_.to_csv('gne_mat_init.csv') # Deu uma leve arrumadinha

In [423]:
# drop duplicated genes 
df_gene_mat = df_.drop_duplicates
# df_gene_mat

# Gene Atribute Matrix

In [385]:
#Split Enrichr ID
df_id = pd.DataFrame(df[0].str.split('_').tolist())
df_id = df_id.rename(columns={0: 'GeneSym', 1: 'Cell', 2:'Assembly'})
df_id.head(3)

,GeneSym,Cell,Assembly
0,CEBPB,myocyte,mm9
1,MYOG,myocyte,mm9
2,MYOG,myocyte,mm9


In [386]:
# Keep only TF
df_tf = df_id["GeneSym"]

In [329]:
# Drop enrichr_id
df_ne = df.drop(df.columns[0], axis=1)

# COMECO

In [334]:
# Gene Attribute Matrix

In [ ]:
dfList = []

files= os.listdir("/Users/Carol/Desktop/annotatedPeaks")

for file in files:
    fname = "/Users/Carol/Desktop/annotatedPeaks/" + file
#     df = pd.read_csv(fname, sep='\t', header = None, names = ['GeneName'])
    df = pd.read_csv(fname, sep='\t', header = 0)
    df = df['GeneName']
#     df = df.assign(file = file.replace('.txt', ''), num = 1)
    dfList.append(df)
    print dfList
finaldf = pd.concat(dfList)
# print finaldf
# PIVOT (LONG TO WIDE)
finaldf = finaldf.pivot_table(index=['GeneName'], columns=[fname], values='num', aggfunc='count').fillna(0).reset_index()    

# CONVERT TO INTEGER
numcols = list(range(1,len(finaldf.columns)))
f_df = finaldf.ix[:,numcols] = finaldf.ix[:,numcols].astype(int)
f_df.to_csv('gene_matrix_mm9.csv')

In [379]:
# Gene Attribute Edges

# Attribute List
List of atributes in the processed dataset

In [424]:
#df_tf

In [ ]:
dfList = []

files= os.listdir("/Users/Carol/Desktop/annotatedPeaks")

for file in files:
    fname = "/Users/Carol/Desktop/annotatedPeaks/" + file
#     df = pd.read_csv(fname, sep='\t', header = None, names = ['GeneName'])
    df = pd.read_csv(fname, sep='\t', header = 0)
    df = df['GeneName']
#     df = df.assign(file = file.replace('.txt', ''), num = 1)
    dfList.append(df)
    print dfList
finaldf = pd.concat(dfList)
# print finaldf
# PIVOT (LONG TO WIDE)
finaldf = finaldf.pivot_table(index=['GeneName'], columns=[fname], values='num', aggfunc='count').fillna(0).reset_index()    

# CONVERT TO INTEGER
numcols = list(range(1,len(finaldf.columns)))
f_df = finaldf.ix[:,numcols] = finaldf.ix[:,numcols].astype(int)
f_df.to_csv('gene_matrix_mm9.csv')

In [ ]:
# Dessa forma um arquivo matrix e' criado, com cada arquivo como uma coluna
# if for some reason not finding GeneName on index, delete DS store file from folder
import os

files = os.listdir("/Users/Carol/Desktop/annotatedPeaks")
# print files

df_ = pd.DataFrame()

for f in files:
    fname = "/Users/Carol/Desktop/annotatedPeaks/"+f
    for f in f:
        df = pd.read_csv(fname, sep='\t', header=0)
        df = df[['GeneName']]
        df= df.rename(columns = {'GeneName':fname})
#         df = df.T
    df_ = pd.concat([df_, df], axis=1)

df_.to_csv('gne_mat_init_stack1.csv') # Deu uma leve arrumadinha

In [2]:
dfList = []

files= os.listdir("/Users/Carol/Desktop/annotatedPeaks/")
print files

for file in files:
    df = pd.read_csv(file, sep='\t', header = None, names = ['GeneName'])        
#     df = df.assign(file = file.replace('.txt', ''), num = 1)
    dfList.append(df)

finaldf = pd.concat(dfList)

# PIVOT (LONG TO WIDE)
finaldf = finaldf.pivot_table(index=['GeneName'], columns=['file'], 
                              values='num', aggfunc='count').fillna(0).reset_index()    
# CONVERT TO INTEGER
numcols = list(range(1,len(finaldf.columns)))
finaldf.ix[:,numcols] = finaldf.ix[:,numcols].astype(int)

NameError: name 'os' is not defined